Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [13]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
import geopy.distance
from dataclasses import dataclass
from icecream import ic
import functools

logging.basicConfig(level=logging.DEBUG)

## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [355]:
vanuatu_cities = pd.read_csv('vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
vanuatu_cities.head()


,name,lat,lon
0,Isangel,-19.53,169.28
1,Lakatoro,-16.09,167.40
2,Longana,-15.30,168.00
3,Luganville,-15.51,167.15
4,Norsup,-16.07,167.39


In [356]:
def distance_matrix(cities):
    dist_matrix = np.zeros((len(cities), len(cities)))
    for c1, c2 in combinations(cities.itertuples(), 2):
        dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geopy.distance.geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
    return dist_matrix



Greedy TSP Solution Using Nearest Neighbor Heuristic

In [387]:

def TSP_nearest_neighbor(cities ,debug = False):

    visited = np.full(len(cities), False) # initialized arrey with false
    dist_matrix = distance_matrix(cities)
    dist_copy = dist_matrix.copy()
    city = 0
    visited[city] = True
    tsp = []
    tsp.append(int(city))
    while not np.all(visited): # until all the cities visited
        dist_copy[:, city] = np.inf # change the distance os the city to infinity so it won't picked again
        closest = np.argmin(dist_copy[city]) #the index of the closest city
        if debug == True:
            logging.debug(f"step: {cities.at[city,'name']} -> {cities.at[closest,'name']} ({dist_matrix[city,closest]:.2f}km)")
        
        visited[closest] = True
        city = closest
        tsp.append(int(city))
    if debug == True:
        logging.debug(f"step: {cities.at[tsp[-1],'name']} -> {cities.at[tsp[0],'name']} ({dist_matrix[tsp[-1],tsp[0]]:.2f}km)")
    
    tsp.append(tsp[0])

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += dist_matrix[c1, c2]
    logging.info(f"result of TSP Solution Using Nearest Neighbor: Found a path of {len(tsp)-1} steps, total length {tot_cost:.2f}km")

TSP_nearest_neighbor(vanuatu_cities, True)

DEBUG:root:step: Isangel -> Vila (223.00km)
DEBUG:root:step: Vila -> Lakatoro (206.74km)
DEBUG:root:step: Lakatoro -> Norsup (2.46km)
DEBUG:root:step: Norsup -> Luganville (67.09km)
DEBUG:root:step: Luganville -> Port Olry (52.02km)
DEBUG:root:step: Port Olry -> Longana (105.77km)
DEBUG:root:step: Longana -> Sola (165.49km)
DEBUG:root:step: Sola -> Isangel (652.96km)
INFO:root:result of TSP Solution Using Nearest Neighbor: Found a path of 8 steps, total length 1475.53km


Helper Functions

In [358]:
def is_valid(c1,c2,degrees,dict_segments):
    if degrees[c1] < 2 and degrees[c2] < 2 : #check no more then 2 degrees
        if dict_segments[c1] != dict_segments[c2]:
            return True
    return False


def union_segments(c1, c2, dict_segments, degrees): #union two segments to one
    leader1 = dict_segments[c1]
    leader2 = dict_segments[c2]
    degrees[c1] += 1
    degrees[c2] += 1
    # If the leaders are different, merge the segments
    if leader1 != leader2:
        # Update all cities that have leader1 as their leader to leader2
        for city in dict_segments:
            if dict_segments[city] == leader1:
                dict_segments[city] = leader2


Greedy TSP Solution Using Segments

In [380]:

def TSP_segments (cities, debug=False):
    # Initialize data structures using indices
    segments = [{idx} for idx in cities.index]
    dict_segments = {idx: idx for idx in cities.index}
    degrees = {idx: 0 for idx in cities.index}
    TSP = []
    counter = len(cities.index)
    steps = 0

    # Calculate and sort all distances between city pairs
    distances = []
    for city1, city2 in combinations(cities.itertuples(index=True), 2):
        distance = geopy.distance.geodesic((city1.lat, city1.lon), (city2.lat, city2.lon)).km
        distances.append((city1.Index, city2.Index, distance))
    distances.sort(key=lambda x: x[2])

    # Build the TSP path
    for edge in distances:
        c1, c2, dist = edge
        steps += 1
        if counter == 1:
            if degrees[c1] < 2 and degrees[c2] < 2:
                TSP.append(edge)
                break
        if is_valid(c1, c2, degrees, dict_segments):
            union_segments(c1, c2, dict_segments, degrees)
            TSP.append(edge)
            counter -= 1

    # Traverse the path and print results
    start_city = TSP[0][1]
    if debug == True:
        print(f"{cities.loc[TSP[0][0], 'name']} -> {cities.loc[TSP[0][1], 'name']} ({TSP[0][2]:.2f} km)")
    total_cost = TSP[0][2]
    TSP.remove(TSP[0])

    while TSP:
        for edge in TSP: #calculate the cost
            c1, c2, dist = edge
            if c1 == start_city:
                total_cost += dist
                if debug == True:
                    print(f"{cities.loc[c1, 'name']} -> {cities.loc[c2, 'name']} ({dist:.2f} km)")
                start_city = c2
                TSP.remove(edge)
                break
            elif c2 == start_city:
                total_cost += dist
                if debug == True:
                    print(f"{cities.loc[c2, 'name']} -> {cities.loc[c1, 'name']} ({dist:.2f} km)")
                start_city = c1
                TSP.remove(edge)
                break

    print(f"result of Greedy TSP Solution Using Segments: Found a path in {steps} steps, total length is: {total_cost:.2f} km")

TSP_segments (vanuatu_cities, True)


Lakatoro -> Norsup (2.46 km)
Norsup -> Luganville (67.09 km)
Luganville -> Port Olry (52.02 km)
Port Olry -> Longana (105.77 km)
Longana -> Sola (165.49 km)
Sola -> Isangel (652.96 km)
Isangel -> Vila (223.00 km)
Vila -> Lakatoro (206.74 km)
result of Greedy TSP Solution Using Segments: Found a path in 28 steps, total length is: 1475.53 km


EA TSP- Helper Functions

In [432]:
rng = np.random.default_rng()

# Function to create an initial population randomly
def create_population(population_size,index_list):
    population = [rng.permutation(index_list) for i in range (population_size)]
    return population


# Function to calculate the total cost (distance) of a TSP route
def tsp_cost(tsp,dist_matrix):
    tot_cost = 0
    for c1 in range (len(tsp)-1):
        tot_cost += dist_matrix[tsp[c1], tsp[c1+1]]

    tot_cost += dist_matrix[tsp[0], tsp[-1]]
    return tot_cost
            

# choose the individual with the lowest cost
def tournament_selection(population,tournament_size,dist_matrix):
    select_parents = np.random.choice(len(population), tournament_size, replace=False)
    tournament = [population[i] for i in select_parents]
    best = sorted(tournament, key = lambda route: tsp_cost(route, dist_matrix))
    return best[0], best[1]


# Helper function to find the index of a specific gene within a parent
def index_np (parent, gene):
    return np.where(parent == gene)[0][0]


# Inver-Over Crossover function
def inver_over_xover(parent1, parent2):
    parent3 = parent1.copy()
    first_index = rng.integers(0, len(parent1)-1)
    parent3 = np.concatenate((parent3[first_index:] , parent3[:first_index]))
    first_index = 0
    first_gene = parent3[first_index]  # Select the first gene of parent1
    
    # Find the position of `first_gene` in `parent2` and get the next gene in `parent2`
    first_index_in_parent2 = index_np(parent2,first_gene)
    second_index_in_parent2 = (first_index_in_parent2 + 1) % len(parent2)
    second_gene = parent2[second_index_in_parent2]
    
    # Define the segment in parent1 to be reversed
    start_segment = first_index + 1
    end_segment = index_np(parent3,second_gene)
        
    # Assemble the offspring by concatenating the different parts    
    segment = parent3[start_segment:end_segment]
    reverse_segment = np.flip(segment)
    offspring = np.concatenate((parent3[0:1], parent2[second_index_in_parent2:second_index_in_parent2+1], reverse_segment, parent3[end_segment+1:]))
    
    return offspring

def inversion_mutation(route):
    copied_route = route.copy()
    idx1, idx2 = sorted(rng.choice(len(route), size=2, replace=False))
    copied_route[idx1:idx2] = copied_route[idx1:idx2][::-1]
    return copied_route


# Scramble Mutation function
def scramble_mutation(route, alleles_num):
    copied_route = route.copy()
    start_index = rng.integers(0, len(copied_route))
    end_index = (start_index + alleles_num) % len(copied_route) 

    # Extract the sublist to be scrambled
    if start_index < end_index:
        sub_route = copied_route[start_index:end_index]
    else: # Case where the sublist wraps around the end
        sub_route = np.concatenate((copied_route[start_index:], copied_route[:end_index]))
    permuted_sub_route = rng.permutation(sub_route)

    # Place permuted sub_route back into copied_route in circular fashion
    if start_index < end_index:
        copied_route[start_index:end_index] = permuted_sub_route
    else: 
        copied_route[start_index:] = permuted_sub_route[:len(copied_route) - start_index]
        copied_route[:end_index] = permuted_sub_route[len(copied_route) - start_index:]
    
    return copied_route

def add_rand(population_size, index_list):
    num_new_individuals = int(0.1 * population_size)
    return create_population(num_new_individuals, index_list)

    


EA TSP Algorithm Using Inver Over Crossover and Scramble Mutation

In [436]:
def TSP_EA(cities, generations ,M_probability,population_size,offspring_size):
    dist_matrix = distance_matrix(cities)
    index_list = [i for i in range (len(cities.index))]
    tournament_size = max(int(offspring_size * 0.1), 2)
    alleles_num=5
    elitism_rate=0.05

    population = create_population(population_size, index_list)
    #MODERN GA FLOW
    for generation in range(generations):
        new_population=[]
        sorted_population = sorted(population, key=lambda route: tsp_cost(route, dist_matrix))                        
        if generation % 1000 == 0 :
            print("generation:" , generation, "current best:",tsp_cost(sorted_population[0],dist_matrix))
        # Determine the number of elites
        num_elites = max(1, int(elitism_rate * population_size))
        elites = sorted_population[:num_elites] #to reserve the best
        new_population = elites.copy()
        if generation%100 == 0:
            new_population.extend(add_rand(int(population_size*0.05),index_list))
    
        for iter in range(offspring_size):
            parent1, parent2 = tournament_selection(population, tournament_size,dist_matrix) #parents selection
        #crossover
            offspring1 = inver_over_xover(parent1, parent2)
        #mutation
            if np.random.random() < M_probability:
                offspring1 = scramble_mutation(offspring1,alleles_num)
            if np.random.random() < M_probability:
                offspring1 = inversion_mutation(offspring1)

            new_population.extend([offspring1])
    
        population = new_population
   
    best_route = min(new_population, key = lambda route: tsp_cost(route,dist_matrix)) #the best route
    best_cost = tsp_cost(best_route,dist_matrix)
    
    print(f"result of EA solution: Found a path of {generation + 1} generations, mutation probability is {M_probability} ,\ntotal length is: {best_cost:.2f} km")

TSP_EA(cities = vanuatu_cities, generations = 10, M_probability=0.5,population_size=10, offspring_size = 5)


generation: 0 current best: 1678.8210597332738
result of EA solution: Found a path of 10 generations, mutation probability is 0.5 ,
total length is: 1396.31 km


ITALY

In [441]:
Italy_cities = pd.read_csv('italy.csv', header=None, names=['name', 'lat', 'lon'])

TSP_nearest_neighbor(Italy_cities)
TSP_segments(Italy_cities)
TSP_EA(Italy_cities, generations=10000, M_probability = 0.3, population_size=300, offspring_size = 100)



INFO:root:result of TSP Solution Using Nearest Neighbor: Found a path of 46 steps, total length 4436.03km


result of Greedy TSP Solution Using Segments: Found a path in 770 steps, total length is: 4730.05 km
generation: 0 current best: 15385.48767068297
generation: 1000 current best: 4291.083493264579
generation: 2000 current best: 4291.083493264579
generation: 3000 current best: 4291.083493264579
generation: 4000 current best: 4291.083493264579
generation: 5000 current best: 4291.083493264579
generation: 6000 current best: 4291.083493264579
generation: 7000 current best: 4291.083493264579
generation: 8000 current best: 4291.083493264579
generation: 9000 current best: 4291.083493264579
result of EA solution: Found a path of 10000 generations, mutation probability is 0.3 ,
total length is: 4291.08 km


In [442]:
Russia_cities = pd.read_csv('russia.csv', header=None, names=['name', 'lat', 'lon'])

TSP_nearest_neighbor(Russia_cities)
TSP_segments(Russia_cities)
TSP_EA(Russia_cities ,generations=10000, M_probability = 0.3, population_size=300, offspring_size = 100)



INFO:root:result of TSP Solution Using Nearest Neighbor: Found a path of 167 steps, total length 42334.16km


result of Greedy TSP Solution Using Segments: Found a path in 12890 steps, total length is: 39758.48 km
generation: 0 current best: 290743.74677103997
generation: 1000 current best: 40589.2020353521
generation: 2000 current best: 35720.41645838816
generation: 3000 current best: 35046.37986252997
generation: 4000 current best: 34834.34412111945
generation: 5000 current best: 34803.247571365835
generation: 6000 current best: 34803.247571365835
generation: 7000 current best: 34803.247571365835
generation: 8000 current best: 34803.247571365835
generation: 9000 current best: 34803.247571365835
result of EA solution: Found a path of 10000 generations, mutation probability is 0.3 ,
total length is: 34803.25 km


In [ ]:
US_cities = pd.read_csv('us.csv', header=None, names=['name', 'lat', 'lon'])

TSP_nearest_neighbor(US_cities)
TSP_segments(US_cities)
TSP_EA(US_cities, generations=10000, M_probability = 0.3, population_size=300, offspring_size = 100)

INFO:root:result of TSP Solution Using Nearest Neighbor: Found a path of 326 steps, total length 48050.03km


result of Greedy TSP Solution Using Segments: Found a path in 52915 steps, total length is: 45845.92 km
generation: 0 current best: 575140.9715772812
generation: 1000 current best: 85492.94431914994
generation: 2000 current best: 53978.143650848906
generation: 3000 current best: 48423.30576466782
generation: 4000 current best: 44144.6817039955


In [ ]:
China_cities = pd.read_csv('china.csv', header=None, names=['name', 'lat', 'lon'])

TSP_nearest_neighbor(China_cities)
TSP_segments(China_cities)
TSP_EA(China_cities,generations=10000, M_probability = 0.3, population_size=300, offspring_size = 100)

